# Notebook 2: ERA5 Reanalysis Download and Launch-Time Weather Extraction

## Objective
This notebook retrieves and processes **ERA5 reanalysis data** in order to
characterize the atmospheric and marine conditions present at the time of each
SpaceX launch from Kennedy Space Center and Cape Canaveral.

The primary goal is to construct a physically meaningful, launch-aligned weather
dataset that can be used for machine learning analysis in subsequent notebooks.

---

## Data sources
This notebook uses the following datasets:

- **ERA5 single-level atmospheric reanalysis** (ECMWF)
  - 10 m wind components
  - 2 m temperature
  - Total cloud cover
  - Precipitation fields
  - Mean sea-level pressure

- **ERA5 wave reanalysis**
  - Significant wave height at an offshore point
  - Used as a proxy for booster recovery and landing constraints

- **SpaceX launch timestamps**
  - Produced in Notebook 1
  - Used to select the exact launch hour from ERA5

---

## Temporal and spatial alignment
ERA5 data are retrieved at **hourly resolution** and spatially subset to a small
region around Kennedy Space Center. For each launch:

- The launch time is floored to the nearest hour
- The nearest ERA5 grid point to KSC is selected
- Offshore wave conditions are sampled at a representative ocean location

This approach balances physical realism with computational efficiency.

---

## Practical considerations
Because ERA5 files retrieved through the CDS API may be delivered as compressed
archives, this notebook includes logic to:
- Detect ZIP-compressed files masquerading as NetCDF
- Safely extract and clean the data before ingestion
- Concatenate monthly files into continuous time series

These steps ensure robust, reproducible data ingestion on local machines.

---

## Feature construction
From the merged ERA5 datasets, the following launch-time features are constructed:

- 10 m wind speed (from u/v components)
- 2 m air temperature (converted to °C)
- Total cloud cover (fraction)
- Total and convective precipitation rates
- Mean sea-level pressure
- Significant wave height

All features are aligned exactly to the launch hour.

---

## Output
The final output of this notebook is a cleaned, merged dataset containing:

- One row per SpaceX launch
- Launch metadata
- Corresponding atmospheric and marine conditions

This dataset serves as the primary input for:
- Weather scrub identification (Notebook 3)
- Machine learning inference of launch constraints (Notebook 4)
- Forecast-based launch confidence estimation (Notebook 5)

---

## Limitations
- ERA5 represents a physically consistent reanalysis, not the exact forecast used
  operationally at the time of launch.
- Spatial averaging may smooth localized effects (e.g., convective cells).
- Offshore wave conditions are approximated using a single representative point.

Despite these limitations, ERA5 provides a high-quality, physically consistent
baseline for learning historical launch-weather relationships.


In [1]:
import cdsapi
from pathlib import Path
import zipfile
import shutil
import os

c = cdsapi.Client()

DATA_DIR = Path("data")
ATM_DIR = DATA_DIR / "era5_atmos"
WAVE_DIR = DATA_DIR / "era5_waves"

ATM_DIR.mkdir(exist_ok=True)
WAVE_DIR.mkdir(exist_ok=True)

2025-12-09 22:24:07,712 INFO [2025-12-03T00:00:00Z] To improve our C3S service, we need to hear from you! Please complete this very short [survey](https://confluence.ecmwf.int/x/E7uBEQ/). Thank you.


In [2]:
ATM_VARIABLES = [
    "10m_u_component_of_wind",
    "10m_v_component_of_wind",
    "2m_temperature",
    "total_cloud_cover",
    "total_precipitation",
    "convective_precipitation",
    "mean_sea_level_pressure",
]

KSC_AREA = [30.0, -82.0, 27.0, -79.0]  # [N, W, S, E] 


In [3]:
WAVE_VARIABLES = [
    "significant_height_of_combined_wind_waves_and_swell"
]

ASDS_AREA = [32.0, -80.0, 30.0, -78.0]  # approximate location

In [4]:
def download_era5(year, month, variables, area, target_path):
    """Generic CDS request wrapper."""
    month_str = f"{month:02d}"

    if target_path.exists():
        print("Skipping existing:", target_path)
        return

    print("Requesting:", target_path.name)
    c.retrieve(
        "reanalysis-era5-single-levels",
        {
            "product_type": "reanalysis",
            "variable": variables,
            "year": str(year),
            "month": month_str,
            "day": [f"{d:02d}" for d in range(1, 32)],
            "time": [f"{h:02d}:00" for h in range(24)],
            "area": area,
            "format": "netcdf",
        },
        str(target_path),
    )


In [5]:
years = range(2010, 2025)
months = range(1, 13)

for year in years:
    for month in months:
        atm_file = ATM_DIR / f"atm_{year}_{month:02d}.nc"
        wave_file = WAVE_DIR / f"swh_{year}_{month:02d}.nc"

        download_era5(year, month, ATM_VARIABLES, KSC_AREA, atm_file)
        download_era5(year, month, WAVE_VARIABLES, ASDS_AREA, wave_file)

print("✓ ERA5 downloads complete.")


Skipping existing: data\era5_atmos\atm_2010_01.nc
Skipping existing: data\era5_waves\swh_2010_01.nc
Requesting: atm_2010_02.nc


2025-12-09 22:24:08,491 INFO Request ID is 411db828-ea7a-4e6e-85bf-f87b7a45762b
2025-12-09 22:24:08,643 INFO status has been updated to accepted
2025-12-09 22:27:03,191 INFO status has been updated to successful
Recovering from connection error [('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))], attempt 1 of 500
Retrying in 120 seconds


Requesting: swh_2010_02.nc


2025-12-09 22:29:35,357 INFO Request ID is f2f54943-57d7-4d03-968a-8355f3d75143
2025-12-09 22:29:35,529 INFO status has been updated to accepted
2025-12-09 22:29:57,365 INFO status has been updated to running
2025-12-09 22:30:08,904 INFO status has been updated to successful


Requesting: atm_2010_03.nc


...

Requesting: atm_2024_12.nc


2025-12-12 00:37:36,777 INFO Request ID is 45153442-c073-4fc5-9642-94eeda8961b7
2025-12-12 00:37:36,934 INFO status has been updated to accepted
2025-12-12 00:38:53,478 INFO status has been updated to running
2025-12-12 00:43:59,142 INFO status has been updated to successful


Requesting: swh_2024_12.nc


2025-12-12 00:44:02,470 INFO Request ID is c4814fd6-7087-4f96-8992-683316b0aebb
2025-12-12 00:44:02,624 INFO status has been updated to accepted
2025-12-12 00:44:53,234 INFO status has been updated to running
2025-12-12 00:45:57,600 INFO status has been updated to successful
                                                                                      

✓ ERA5 downloads complete.


In [12]:
import zipfile
import shutil
import os
import time
from pathlib import Path

import xarray as xr

DATA_DIR = Path("data")
ATM_DIR = DATA_DIR / "era5_atmos"
WAVE_DIR = DATA_DIR / "era5_waves"

def extract_inner_netcdfs(folder: Path, prefix: str):
    """
    Handle 'zip masquerading as .nc' files produced by CDS.
    For each *.nc:
      - Try to open as Zip.
      - Extract inner .nc members with informative names.
      - Rename the outer container to .zip so xarray won't touch it.
    """
    nc_files = sorted(folder.glob("*.nc"))
    print(f"\n[{folder.name}] Found {len(nc_files)} outer .nc files")

    for f in nc_files:
        print(f"Processing {f.name} ...")
        try:
            with zipfile.ZipFile(f, "r") as zf:
                inner_names = zf.namelist()
                print("  Inner entries:", inner_names)

                # Parse year, month from prefix_YYYY_MM.nc
                stem_parts = f.stem.split("_")  # e.g., ['atm', '2010', '01']
                year = stem_parts[1] if len(stem_parts) > 2 else "xxxx"
                month = stem_parts[2] if len(stem_parts) > 2 else "mm"

                # Extract only .nc members
                for inner in inner_names:
                    if not inner.endswith(".nc"):
                        continue

                    tag = inner.replace(".nc", "").replace("data_stream-", "")
                    out_name = f"{prefix}_{year}_{month}_{tag}.nc"
                    out_path = folder / out_name

                    if out_path.exists():
                        print("    Skipping existing", out_name)
                        continue

                    print("    Extracting", inner, "->", out_name)
                    with zf.open(inner) as src, open(out_path, "wb") as dst:
                        shutil.copyfileobj(src, dst)

            # Rename container so we don't try to open it with xarray
            try:
                renamed = f.with_suffix(".zip")
                f.rename(renamed)
                print("  Renamed container to:", renamed.name)
            except PermissionError:
                time.sleep(0.5)
                try:
                    renamed = f.with_suffix(".zip")
                    f.rename(renamed)
                    print("  Renamed after retry to:", renamed.name)
                except PermissionError:
                    print("  WARNING: Could not rename", f.name, "– leaving as is")

        except zipfile.BadZipFile:
            # Not actually a zip, likely a real NetCDF or an error file
            print("  Not a zipfile; leaving as is:", f.name)

# 1. Extract inner NetCDFs for atmosphere and waves
extract_inner_netcdfs(ATM_DIR, prefix="atm")
extract_inner_netcdfs(WAVE_DIR, prefix="swh")

# 2. Build atmospheric dataset: concatenate instant + accum and merge
inst_files = sorted(ATM_DIR.glob("*stepType-instant*.nc"))
accum_files = sorted(ATM_DIR.glob("*stepType-accum*.nc"))

print("\nInstantaneous atm files:", len(inst_files))
print("Accumulated atm files:", len(accum_files))

inst_datasets = []
for f in inst_files:
    try:
        ds = xr.open_dataset(f, engine="netcdf4")
        inst_datasets.append(ds)
    except Exception as e:
        print("  Failed to open instant:", f.name, "->", e)

accum_datasets = []
for f in accum_files:
    try:
        ds = xr.open_dataset(f, engine="netcdf4")
        accum_datasets.append(ds)
    except Exception as e:
        print("  Failed to open accum:", f.name, "->", e)

atm_inst = xr.concat(inst_datasets, dim="valid_time")
atm_accum = xr.concat(accum_datasets, dim="valid_time")

atm_ds = xr.merge([atm_inst, atm_accum])

# Drop useless 'time' dim if present and rename valid_time → time
if "time" in atm_ds.dims and atm_ds.dims["time"] == 1:
    atm_ds = atm_ds.isel(time=0, drop=True)

atm_ds = atm_ds.rename({"valid_time": "time"})

print("\nAtmospheric dataset:")
print(atm_ds)

# 3. Build wave dataset: concatenate all extracted wave .nc files
wave_files = sorted(WAVE_DIR.glob("*.nc"))
print("\nWave files:", len(wave_files))

wave_datasets = []
for f in wave_files:
    try:
        ds = xr.open_dataset(f, engine="netcdf4")
        wave_datasets.append(ds)
    except Exception as e:
        print("  Failed to open wave file:", f.name, "->", e)

wave_ds = xr.concat(wave_datasets, dim="valid_time")

if "time" in wave_ds.dims and wave_ds.dims["time"] == 1:
    wave_ds = wave_ds.isel(time=0, drop=True)

wave_ds = wave_ds.rename({"valid_time": "time"})

print("\nWave dataset:")
print(wave_ds)



[era5_atmos] Found 182 outer .nc files
Processing atm_2010_01.nc ...
  Inner entries: ['data_stream-oper_stepType-instant.nc', 'data_stream-oper_stepType-accum.nc']
    Extracting data_stream-oper_stepType-instant.nc -> atm_2010_01_oper_stepType-instant.nc
    Extracting data_stream-oper_stepType-accum.nc -> atm_2010_01_oper_stepType-accum.nc
  Renamed container to: atm_2010_01.zip
Processing atm_2010_02.nc ...
  Inner entries: ['data_stream-oper_stepType-instant.nc', 'data_stream-oper_stepType-accum.nc']
    Extracting data_stream-oper_stepType-instant.nc -> atm_2010_02_oper_stepType-instant.nc
    Extracting data_stream-oper_stepType-accum.nc -> atm_2010_02_oper_stepType-accum.nc
  Renamed container to: atm_2010_02.zip
Processing atm_2010_03.nc ...
  Inner entries: ['data_stream-oper_stepType-instant.nc', 'data_stream-oper_stepType-accum.nc']
    Extracting data_stream-oper_stepType-instant.nc -> atm_2010_03_oper_stepType-instant.nc
    Extracting data_stream-oper_stepType-accum.nc 

C:\Users\bolhu\AppData\Local\Temp\ipykernel_20792\458742933.py:101: FutureWarning: In a future version of xarray the default value for compat will change from compat='no_conflicts' to compat='override'. This is likely to lead to different results when combining overlapping variables with the same name. To opt in to new defaults and get rid of these warnings now use `set_options(use_new_combine_kwarg_defaults=True) or set compat explicitly.
  atm_ds = xr.merge([atm_inst, atm_accum])
C:\Users\bolhu\AppData\Local\Temp\ipykernel_20792\458742933.py:101: FutureWarning: In a future version of xarray the default value for compat will change from compat='no_conflicts' to compat='override'. This is likely to lead to different results when combining overlapping variables with the same name. To opt in to new defaults and get rid of these warnings now use `set_options(use_new_combine_kwarg_defaults=True) or set compat explicitly.
  atm_ds = xr.merge([atm_inst, atm_accum])



Atmospheric dataset:
<xarray.Dataset> Size: 629MB
Dimensions:    (time: 132240, latitude: 13, longitude: 13)
Coordinates:
  * time       (time) datetime64[ns] 1MB 2010-01-01 ... 2010-01-31T23:00:00
  * latitude   (latitude) float64 104B 30.0 29.75 29.5 29.25 ... 27.5 27.25 27.0
  * longitude  (longitude) float64 104B -82.0 -81.75 -81.5 ... -79.25 -79.0
    number     int64 8B 0
    expver     (time) <U4 2MB '0001' '0001' '0001' ... '0001' '0001' '0001'
Data variables:
    u10        (time, latitude, longitude) float32 89MB 1.782 1.4 ... -6.739
    v10        (time, latitude, longitude) float32 89MB 2.008 1.944 ... -6.756
    t2m        (time, latitude, longitude) float32 89MB 291.7 291.6 ... 293.3
    tcc        (time, latitude, longitude) float32 89MB 0.9005 0.925 ... 0.9517
    msl        (time, latitude, longitude) float32 89MB 1.018e+05 ... 1.018e+05
    tp         (time, latitude, longitude) float32 89MB 3.004e-05 ... 7.153e-06
    cp         (time, latitude, longitude) float32 8

In [13]:
import numpy as np
import pandas as pd
from pathlib import Path

DATA_DIR = Path("data")

# ------------------------------------------------------
# 1. Select grid points
# ------------------------------------------------------

# KSC launch pad
KSC_LAT, KSC_LON = 28.5, -80.6
if atm_ds.longitude.max() > 180:
    ksc_lon = (KSC_LON + 360) % 360
else:
    ksc_lon = KSC_LON

atm_point = atm_ds.sel(latitude=KSC_LAT, longitude=ksc_lon, method="nearest")

# Approx droneship location
DRONE_LAT, DRONE_LON = 31.0, -79.0
if wave_ds.longitude.max() > 180:
    drone_lon = (DRONE_LON + 360) % 360
else:
    drone_lon = DRONE_LON

wave_point = wave_ds.sel(latitude=DRONE_LAT, longitude=drone_lon, method="nearest")

# ------------------------------------------------------
# 2. Convert to DataFrame and align time to hour
# ------------------------------------------------------

atm_df = atm_point.to_dataframe().reset_index()
atm_df["time_hour"] = pd.to_datetime(atm_df["time"]).dt.floor("h")

wave_df = wave_point.to_dataframe().reset_index()
wave_df["time_hour"] = pd.to_datetime(wave_df["time"]).dt.floor("h")

print("Atmospheric DF shape:", atm_df.shape)
print("Wave DF shape:", wave_df.shape)

# Identify wave variable name (swh)
wave_var_candidates = [v for v in wave_df.columns if v not in ["time", "latitude", "longitude", "number", "expver", "time_hour"]]
if "swh" in wave_df.columns:
    wave_var = "swh"
elif wave_var_candidates:
    wave_var = wave_var_candidates[0]
else:
    raise RuntimeError("Could not find wave height variable in wave_df")

# ------------------------------------------------------
# 3. Load SpaceX launches and align timestamps
# ------------------------------------------------------

spacex = pd.read_csv(
    DATA_DIR / "spacex_launches_ccafs_2010_2024.csv",
    parse_dates=["date_utc"]
)

spacex["date_hour"] = spacex["date_utc"].dt.floor("h")

# Make timezone-naive for alignment with ERA5 (UTC but naive)
if spacex["date_hour"].dt.tz is not None:
    spacex["date_hour_utc_naive"] = (
        spacex["date_hour"]
        .dt.tz_convert("UTC")
        .dt.tz_localize(None)
    )
else:
    spacex["date_hour_utc_naive"] = spacex["date_hour"]

launch_hours = spacex["date_hour_utc_naive"].dropna().drop_duplicates()

print("Unique launch hours:", len(launch_hours))

# Filter ERA5 to launch hours
atm_launch = atm_df[atm_df["time_hour"].isin(launch_hours)].copy()
wave_launch = wave_df[wave_df["time_hour"].isin(launch_hours)].copy()

print("Atmosphere at launch hours:", len(atm_launch))
print("Waves at launch hours:", len(wave_launch))

# ------------------------------------------------------
# 4. Engineer weather & wave features
# ------------------------------------------------------

# Wind speed
atm_launch["wind_speed_10m"] = np.hypot(
    atm_launch["u10"], atm_launch["v10"]
)

# Temperature in °C
atm_launch["t2m_C"] = atm_launch["t2m"] - 273.15

# Cloud fraction
atm_launch["tcc_frac"] = atm_launch["tcc"]

# Precip: m -> mm/hr
atm_launch["tp_mmhr"] = atm_launch["tp"] * 1000.0
atm_launch["cp_mmhr"] = atm_launch["cp"] * 1000.0

# Keep & rename needed cols
atm_final = atm_launch[
    ["time_hour", "wind_speed_10m", "t2m_C", "tcc_frac",
     "tp_mmhr", "cp_mmhr", "msl"]
].drop_duplicates("time_hour")

wave_final = wave_launch[["time_hour", wave_var]].drop_duplicates("time_hour")
wave_final = wave_final.rename(columns={wave_var: "swh"})

# ------------------------------------------------------
# 5. Merge SpaceX + Atmosphere + Waves
# ------------------------------------------------------

merged = (
    spacex
    .merge(atm_final, left_on="date_hour_utc_naive", right_on="time_hour", how="left")
    .merge(wave_final, on="time_hour", how="left")
)

# Drop any rows that still don't have weather or wave data
merged_clean = merged.dropna(subset=["wind_speed_10m", "swh"]).copy()

print("Final merged shape:", merged_clean.shape)
merged_clean.head()

# ------------------------------------------------------
# 6. Save for the ML notebook
# ------------------------------------------------------

out_path = DATA_DIR / "spacex_era5_ksc_droneship_2010_2024.csv"
merged_clean.to_csv(out_path, index=False)
print("Saved merged ML dataset to:", out_path)


Atmospheric DF shape: (132240, 13)
Wave DF shape: (131496, 7)
Unique launch hours: 154
Atmosphere at launch hours: 154
Waves at launch hours: 154
Final merged shape: (154, 19)
Saved merged ML dataset to: data\spacex_era5_ksc_droneship_2010_2024.csv
